<a href="https://colab.research.google.com/github/andrkech/GENERATIVE-METHODS-IN-GENOMICS/blob/main/VARIANT_READS_GENERATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set up

In [ ]:
!pip install -q biopython
import os
import random
from collections import Counter, defaultdict
import csv
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.5 MB/s eta 0:00:00


### Hyperparameters

In [ ]:
FASTA_DIR = "/content/drive/MyDrive/BIOINFORMATICS/THESIS_KECHAGIAS/DATA/INFO/TP53.fasta"
VARIANT_DIR = "/content/drive/MyDrive/BIOINFORMATICS/THESIS_KECHAGIAS/DATA/INFO/totalVariantDataGATK_4_1_0_0.csv"

READ_SIZE = 300
NUM_READS = 10000
FASTA_START = 7571720
SEED = 2 # a seed for both the reads generation and the variant application

READS_DIR = "/content/drive/MyDrive/BIOINFORMATICS/THESIS_KECHAGIAS/VARIANT_READS"
timestamp = datetime.now().strftime("%m%d")

### Read FASTA file

In [ ]:
def read_fasta(file_path):
    sequences = []
    try:
        for record in SeqIO.parse(file_path, "fasta"):
            sequences.append(str(record.seq))

    except Exception as e:
        print(f"Error: {e}")
        return None

    return ''.join(sequences)

In [ ]:
fasta_sequence = read_fasta(FASTA_DIR)

print(fasta_sequence, "\n", len(fasta_sequence))

CTAGAGCCACCGTCCAGGGAGCAGGTAGCTGCTGGGCTCCGGGGACACTTTGCGTTCGGGCTGGGAGCGTGCTTTCCACGACGGTGACACGCTTCCCTGGATTGGGTAAGCTCCTGACTGAACTTGATGAGTCCTCTCTGAGTCACGGGCTCTCGGCTCCGTGTATTTTCAGCTCGGGAAAATCGCTGGGGCTGGGGGTGGGGCAGTGGGGACTTAGCGAGTTTGGGGGTGAGTGGGATGGAAGCTTGGCTAGAGGGATCATCATAGGAGTTGCATTGTTGGGAGACCTGGGTGTAGATGATGGGGATGTTAGGACCATCCGAACTCAAAGTTGAACGCCTAGGCAGAGGAGTGGAGCTTTGGGGAACCTTGAGCCGGCCTAAAGCGTACTTCTTTGCACATCCACCCGGTGCTGGGCGTAGGGAATCCCTGAAATAAAAGATGCACAAAGCATTGAGGTCTGAGACTTTTGGATCTCGAAACATTGAGAACTCATAGCTGTATATTTTAGAGCCCATGGCATCCTAGTGAAAACTGGGGCTCCATTCCGAAATGATCATTTGGGGGTGATCCGGGGAGCCCAAGCTGCTAAGGTCCCACAACTTCCGGACCTTTGTCCTTCCTGGAGCGATCTTTCCAGGCAGCCCCCGGCTCCGCTAGATGGAGAAAATCCAATTGAAGGCTGTCAGTCGTGGAAGTGAGAAGTGCTAAACCAGGGGTTTGCCCGCCAGGCCGAGGAGGACCGTCGCAATCTGAGAGGCCCGGCAGCCCTGTTATTGTTTGGCTCCACATTTACATTTCTGCCTCTTGCAGCAGCATTTCCGGTTTCTTTTTGCCGGAGCAGCTCACTATTCACCCGATGAGAGGGGAGGAGAGAGAGAGAAAATGTCCTTTAGGCCGGTTCCTCTTACTTGGCAGAGGGAGGCTGCTATTCTCCGCCTGCATTTCTTTTTCTGGATTACTTAGTTATGGCCTTTGCAAAGGCAGGGGTATTTGTTTT

### Generate Reads.

In [ ]:
def generate_reads(sequence, read_size, num_reads, seed):
    if seed is not None:
        random.seed(seed)

    reads_with_positions = []
    seq_len = len(sequence)

    for _ in range(num_reads):
        if not seq_len <= read_size:
            start = random.randint(0, seq_len - read_size)  # Random start position ensuring read fits within sequence
        else:
            return ""

        read = sequence[start:start + read_size]
        reads_with_positions.append((read, start))

    return reads_with_positions

In [ ]:
synthetic_reads = generate_reads(fasta_sequence, READ_SIZE, NUM_READS, SEED)

for read, position in synthetic_reads[:10]:
    print(f"Read: {read} \nstarting from position: {position}")

Read: ACTAAAAAATACAAAAATTAGCTGGGCGTGGTGGGTGCCTGTAATCCCAGCTATTCGGGAGGGTGAGGCAGGAGAATCGCTTGAACCCGGGAGGCAGAGGTTGCAGTGAGCCAAGATCGTGCCACTACACTCCAGCCTGGGCGACAAGAACGAAACTCCGTCTCAAAAAAAAGGGGGGAATCATACATTATGTGCTCATTTTTGTCGGGCTTCTGTCCTTCAATGTACTGTCTGACATTCGTTCATGTTGTATATATCAGTATTTTGCTCCTTTTCATTTAGTATAGTCCATCGATTGTA 
starting from position: 1853
Read: CTCTACTGAATGCTTTTAATTTTAATTATTTTACAGTTGGAGTATAGGGCTACCATTTTAGTGCTATTTTCTTTTTTTCTTTGTTAATTTTTGAGACAGGGACTCACACTGTTGCCCAGGCTAGAGTACAATGGCACAATCAAGGCTTACTGCAGCCTCGAACCCCTGGGCTCAAGCAGTCCTCTAGCAGCCTCACGAGTAGCTGGGATTACTCCACCACACCCAGCTAACTATTTTATTTTTTTGTATTGACAGGATCTCACTATGTTGCCCAGGCTGGTCTCAAACTGCTGGCCTCAA 
starting from position: 3001
Read: CATCTTTTTTTTTTTTTTTAACCCCAGGGTCATGAAGATATTATCTTACATTTTCTTTTAGGACCTTTATGGTTGTAAGTTTTACAGTAAGGTCCTTGAGCCATTAATTAATTCTTAAAATTAATTGTTTATGGTGTGAGGTGTAGGAGTCAGTCTCTGGTATCTTTCCTGTATGGAAATCCAGTTATTCTGTCTCCACTTGTTGAAATAGGCTTCCTTTCTCTACTGAATGCTTTTAATTTTAATTATTTTACAGTTGGAGTATAGGGCTACCATTTTAGTGCTATTTTCTTTTTTTCT 
starting from posi

### Process Variant List.

In [ ]:
def read_and_process_csv(file_path):
    try:
        # Initialize the list for processed data
        processed_list = []

        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                line = row[0]

                # Split the line by `,` avoiding quoted commas
                elements = []
                current = []
                in_quotes = False

                for char in line:
                    if char == '"' and (not current or current[-1] != '\\'):
                        in_quotes = not in_quotes
                        current.append(char)

                    elif char == ',' and not in_quotes:
                        elements.append(''.join(current).strip('"'))
                        current = []
                    else:
                        current.append(char)

                elements.append(''.join(current).strip('"'))

                # Replace empty strings with ''
                elements = [elem if elem else '' for elem in elements]

                processed_list.append(elements)

        processed_array = np.array(processed_list, dtype=object)
        return processed_array

    except Exception as e:
        print(f"Error: {e}")

        return None

In [ ]:
try:
    processed_list = read_and_process_csv(CSV_DIR)

    for i, row in enumerate(processed_list):
        print(row)
        if len(row) != len(processed_list[0]):
            raise ValueError(f"Possible error at row {i+1}. Length of row is {len(row)}, expected {len(processed_list[0])}. The total amount of data should be equal for each variant.")

except FileNotFoundError:
    raise FileNotFoundError("Failed to read and process the CSV file.")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

['SampleID' 'Gene.refGene' 'CHROM' 'POS' 'REF' 'ALT' 'Func.refGene'
 'ExonicFunc.refGene' 'AAChange.refGene' 'CLNDN' 'CLNDISDB' 'CLNREVSTAT'
 'CLNSIG' 'CLNALL' 'cosmic70' 'DP' 'gt_TDP' 'gt_TAD' 'RefCoverage'
 'AltCoverage1' 'Ref.Median.Base.Quality' 'Alt.Median.Base.Quality1'
 'Ref.Median.Mapping.Quality' 'Alt.Median.Mapping.Quality1' 'Genotype'
 'VAF1' 'CovRepAF1' 'VAF1.CovRepAF1_diff']
['E22242_S1' 'TP53' 'chr17' '7573897' 'T' 'A' 'intronic' '' ''
 'none_provided' 'MedGen:CN235283' 'criteria_provided,_single_submitter'
 'Benign' 'Benign:none_provided'
 'ID\\x3dCOSM45239\\x3bOCCURENCE\\x3d1(upper_aerodigestive_tract)' '110'
 '40682' '20820, 19829' '57' '53' '36' '32' '60' '60' '0/1' '0.474'
 '0.4874' '-0.0134']
['E22242_S1' 'TP53' 'chr17' '7576912' 'T' 'G' 'exonic' 'nonsynonymous_SNV'
 'TP53:NM_000546:exon9:c.A934C:p.T312P' '' '' '' '' '' '' '426' '275914'
 '256092, 18220' '330' '49' '27' '14' '60' '60' '0/1' '0.072' '0.066'
 '0.00599999999999999']
['E22242_S1' 'TP53' 'chr17' '7576914

### Extract SNPs.

In [ ]:
def extract_snps(variants_list):
    snp_list = []
    ref_index = 4
    alt_index = 5

    for row in variants_list:
        ref_nucleotide = row[ref_index]
        alt_nucleotide = row[alt_index]

        if len(ref_nucleotide) == 1 and len(alt_nucleotide) == 1:
            snp_list.append(row)

    return snp_list

In [ ]:
SNPs = extract_snps(processed_list)

for SNP in SNPs:
    print(f"Variant position: {int(SNP[3])}, reference base: {SNP[4]}, alteration: {SNP[5]}, frequency: {float(SNP[-3])}")

Variant position: 7573897, reference base: T, alteration: A, frequency: 0.474
Variant position: 7576912, reference base: T, alteration: G, frequency: 0.072
Variant position: 7576914, reference base: T, alteration: G, frequency: 0.039
Variant position: 7577051, reference base: T, alteration: G, frequency: 0.012
Variant position: 7578161, reference base: C, alteration: A, frequency: 0.066
Variant position: 7578313, reference base: A, alteration: C, frequency: 0.052
Variant position: 7578467, reference base: T, alteration: G, frequency: 0.048
Variant position: 7579409, reference base: A, alteration: C, frequency: 0.085
Variant position: 7579472, reference base: G, alteration: C, frequency: 0.993
Variant position: 7579789, reference base: T, alteration: G, frequency: 0.207
Variant position: 7579801, reference base: G, alteration: C, frequency: 0.967


### Generate a synthetic variants list. (Optional)

This module is used only in the case that the variant list does not accord with the FASTA file. (READ_VARIANTS = False)

In [ ]:
def generate_synthetic_variants(SNPs, sequence):
    # Create a dictionary mapping reference nucleotides to lists of (alt, VAF) pairs
    ref_alt_vaf = defaultdict(list)
    for snp in SNPs:
        ref_alt_vaf[snp[4]].append((snp[5], float(snp[-3])))  # Store alt and VAF for each ref

    # Count occurrences of each ref nucleotide in SNPs
    ref_counts = Counter(var[4] for var in SNPs)

    # Efficiently build matching_indices using a dictionary
    matching_indices = defaultdict(list)
    for pos, nucl in enumerate(sequence):
        if nucl in ref_counts:  # Nucleotides in SNPs
            matching_indices[nucl].append(pos)  # Group positions by nucleotide

    synthetic_variants = []
    for ref_nucleotide, count in ref_counts.items():
        # Sample the required number of positions for each ref nucleotide
        try:
            positions = random.sample(matching_indices[ref_nucleotide], count)
        except ValueError:
            # If not enough positions are available, take all of them
            positions = matching_indices[ref_nucleotide]

        for fasta_position in positions:  # Use fasta_position directly
            # Get alt and VAF from the original SNP
            alt_nucleotide, vaf = random.choice(ref_alt_vaf[ref_nucleotide])
            synthetic_variants.append((fasta_position + 1, ref_nucleotide, alt_nucleotide, vaf))  # +1 to make position 1-based

    return synthetic_variants

In [ ]:
if REAL_VARIANTS:
    # Create a list with the needed information
    variant_list = []

    for SNP in SNPs:
        variant_list.append((int(SNP[3]) - fasta_start, SNP[4], SNP[5], float(SNP[-3])))

    print(*variant_list, sep="\n")
else:
    # Generate synthetic variants mimicing the original
    variant_list = generate_synthetic_variants(SNPs, fasta_sequence)

    for variant in variant_list:
        print(f"Variant: {variant}")

(2177, 'T', 'A', 0.474)
(5192, 'T', 'G', 0.072)
(5194, 'T', 'G', 0.039)
(5331, 'T', 'G', 0.012)
(6441, 'C', 'A', 0.066)
(6593, 'A', 'C', 0.052)
(6747, 'T', 'G', 0.048)
(7689, 'A', 'C', 0.085)
(7752, 'G', 'C', 0.993)
(8069, 'T', 'G', 0.207)
(8081, 'G', 'C', 0.967)


### Apply Variants.

In [ ]:
def apply_variants(reads_with_positions, snp_list, fasta_start, shuffle_seed=None):
    modified_reads = []
    for read, start_pos in reads_with_positions:
        new_read = list(read)
        has_variant = False
        for variant_pos, ref_base, alt_base, vaf in snp_list:
            read_pos = variant_pos - int(start_pos)

            # Check if the variant is within this read and matches the reference base
            if 0 <= read_pos < len(read) and read[read_pos - 1] == ref_base:
                # Determine if this read should have the variant applied
                should_modify = random.random() < vaf if shuffle_seed is None else \
                                random.Random(shuffle_seed + variant_pos).random() < vaf

                if should_modify:
                    new_read[read_pos] = alt_base
                    has_variant = True

                    print(f"Variant {ref_base} --> {alt_base} applied in {read_pos - 1} of read {read}")

        modified_reads.append((
            "".join(new_read),
            start_pos,
            has_variant
        ))

    return modified_reads

In [ ]:
variant_reads = apply_variants(synthetic_reads, variant_list, FASTA_START, shuffle_seed=SEED)

Variant T --> A applied in 251 of read AGAATCGCTTGAACCCGGGAGGCAGAGGTTGCAGTGAGCCAAGATCGTGCCACTACACTCCAGCCTGGGCGACAAGAACGAAACTCCGTCTCAAAAAAAAGGGGGGAATCATACATTATGTGCTCATTTTTGTCGGGCTTCTGTCCTTCAATGTACTGTCTGACATTCGTTCATGTTGTATATATCAGTATTTTGCTCCTTTTCATTTAGTATAGTCCATCGATTGTATATCCGTCCTTTTGATGGCCTTTTGAGTTGTTTCCCATTTGCGGTTATGAAATAAAGCTGCTATAAACATTC
Variant G --> C applied in 58 of read TGCTTGTAGTCCTAGCTACTCAGCAGGCTGAGGCAGGAGAATCATTTGAATCCGGGAGGAGGTTGCAGTAAGCGGAGATAGTGCCACTGTACTCCAGCCTGGGCAATAAGAGCTGAGACTCCGTCTCAAAATAAAATAAAATAAAATAAAATAAAATAAAATAAAATAAAAAAAGAAAAGAGCCTGCCATTAAAGGAGCTGTTTGGTAGGGGATGTTTTGTCAGTGCAAACAACAGAAAAGTGGGCTGGGCACAGTGGTTCATGCCTGTAATCCCAGCACTTTGGGAGGCCAAGGCGGGC
Variant T --> A applied in 198 of read TACACTCCAGCCTGGGCGACAAGAACGAAACTCCGTCTCAAAAAAAAGGGGGGAATCATACATTATGTGCTCATTTTTGTCGGGCTTCTGTCCTTCAATGTACTGTCTGACATTCGTTCATGTTGTATATATCAGTATTTTGCTCCTTTTCATTTAGTATAGTCCATCGATTGTATATCCGTCCTTTTGATGGCCTTTTGAGTTGTTTCCCATTTGCGGTTATGAAATAAAGCTGCTATAAACATTCTTGTACAATTCTTTTTGTGATCATATGTTTTCGTG

### Save Variant Reads.

In [ ]:
df_variant_reads = pd.DataFrame(variant_reads, columns=["Read", "Start_position", "Has_Variant"])
df_variant_reads['Has_Variant'] = df_variant_reads['Has_Variant'].astype(str)

output_csv_path = os.path.join(save_reads_dir, f"variant_reads_{READ_SIZE}_{NUM_READS}_{timestamp}.csv")
df_variant_reads.to_csv(output_csv_path, index=False)